# Імпортування даних у таблиці фактів сховища даних

Імпортуємо бібліотеки

In [2]:
import pandas as pd
from dotenv import dotenv_values
from sqlalchemy import create_engine, text
import os

Завантажуємо змінні оточення з .env файлу для з'єднання зі сховищем

In [3]:
config = dotenv_values()

DB_NAME = config.get('DB_NAME')
DB_USER = config.get('DB_USER')
DB_HOST = config.get('DB_HOST')
DB_PASSWORD = config.get('DB_PASSWORD')
DB_PORT = config.get('DB_PORT')

З'єднуємося з базою даних

In [4]:
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

Функція для внесення даних в таблицю сховища

In [7]:
def insert_data(df, table):
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT COUNT(*) FROM {table}"))
        count = result.scalar()

        if count == 0:
            df.to_sql(table, engine, if_exists='append', index=False)

Завантажимо датафрейми з ідентифікаторами сховища та датасетів

In [9]:
circuits_id_df = pd.read_csv('../adapted_data/circuitsId.csv')
constructors_id_df = pd.read_csv('../adapted_data/constructorsId.csv')
countries_id_df = pd.read_csv('../adapted_data/countriesId.csv')
drivers_id_df = pd.read_csv('../adapted_data/driversId.csv')
grand_prix_id_df = pd.read_csv('../adapted_data/grand_prixId.csv')
seasons_id_df = pd.read_csv('../adapted_data/seasonsId.csv')

circuits_id_df.head()

,id,dbId
0,adelaide,1
1,aida,2
2,ain-diab,3
3,aintree,4
4,anderstorp,5


## Races FP1 Results Table

In [5]:
races_fp1_results_df = pd.read_csv('../data/f1db-races-free-practice-1-results.csv')
races_fp1_results_df.head()

,raceId,year,round,positionDisplayOrder,positionNumber,positionText,driverNumber,driverId,constructorId,engineManufacturerId,tyreManufacturerId,time,timeMillis,gap,gapMillis,interval,intervalMillis,laps
0,435,1986,15,1,1,1,6,nelson-piquet,williams,honda,goodyear,1:18.601,78601.0,NaN,NaN,NaN,NaN,NaN
1,435,1986,15,2,2,2,12,ayrton-senna,lotus,renault,goodyear,1:18.779,78779.0,+0.178,178.0,+0.178,178.0,NaN
2,435,1986,15,3,3,3,20,gerhard-berger,benetton,bmw,pirelli,1:19.004,79004.0,+0.403,403.0,+0.225,225.0,NaN
3,435,1986,15,4,4,4,2,keke-rosberg,mclaren,tag,goodyear,1:19.099,79099.0,+0.498,498.0,+0.095,95.0,NaN
4,435,1986,15,5,5,5,5,nigel-mansell,williams,honda,goodyear,1:19.588,79588.0,+0.987,987.0,+0.489,489.0,NaN


Замінимо ідентифікатори датасету на ті, що згенерувало сховище (driver_id, constructor_id)

In [15]:
races_fp1_results_df['driver_id'] = races_fp1_results_df.merge(drivers_id_df, left_on='driverId', right_on='id', how='left')['dbId']
races_fp1_results_df['constructor_id'] = races_fp1_results_df.merge(constructors_id_df, left_on='constructorId', right_on='id', how='left')['dbId']

Приводимо дані у потрібний нам формат для сховища

In [17]:
races_fp1_results_df = races_fp1_results_df[['raceId', 'driver_id', 'constructor_id', 'positionDisplayOrder', 'positionNumber', 'time', 'timeMillis', 'gap', 'gapMillis', 'interval', 'intervalMillis', 'laps']]
races_fp1_results_df.head()

,raceId,driver_id,constructor_id,positionDisplayOrder,positionNumber,time,timeMillis,gap,gapMillis,interval,intervalMillis,laps
0,435,644,180,1,1,1:18.601,78601.0,NaN,NaN,NaN,NaN,NaN
1,435,68,101,2,2,1:18.779,78779.0,+0.178,178.0,+0.178,178.0,NaN
2,435,316,22,3,3,1:19.004,79004.0,+0.403,403.0,+0.225,225.0,NaN
3,435,521,112,4,4,1:19.099,79099.0,+0.498,498.0,+0.095,95.0,NaN
4,435,652,180,5,5,1:19.588,79588.0,+0.987,987.0,+0.489,489.0,NaN


Перейменуємо стовпці

In [19]:
races_fp1_results_df.columns = ['race_id', 'driver_id', 'constructor_id', 'position_display_order', 'position_number', 'time', 'time_millis', 'gap', 'gap_millis', 'interval', 'interval_millis', 'laps']

Вносимо дані з датафрейму у таблицю сховища

In [20]:
insert_data(races_fp1_results_df, 'races_fp1_results')

## Races FP2 Results Table

In [21]:
races_fp2_results_df = pd.read_csv('../data/f1db-races-free-practice-2-results.csv')
races_fp2_results_df.head()

,raceId,year,round,positionDisplayOrder,positionNumber,positionText,driverNumber,driverId,constructorId,engineManufacturerId,tyreManufacturerId,time,timeMillis,gap,gapMillis,interval,intervalMillis,laps
0,435,1986,15,1,1,1,12,ayrton-senna,lotus,renault,goodyear,1:17.977,77977.0,NaN,NaN,NaN,NaN,NaN
1,435,1986,15,2,2,2,20,gerhard-berger,benetton,bmw,pirelli,1:18.088,78088.0,+0.111,111.0,+0.111,111.0,NaN
2,435,1986,15,3,3,3,19,teo-fabi,benetton,bmw,pirelli,1:18.154,78154.0,+0.177,177.0,+0.066,66.0,NaN
3,435,1986,15,4,4,4,6,nelson-piquet,williams,honda,goodyear,1:18.353,78353.0,+0.376,376.0,+0.199,199.0,NaN
4,435,1986,15,5,5,5,5,nigel-mansell,williams,honda,goodyear,1:18.785,78785.0,+0.808,808.0,+0.432,432.0,NaN


Замінимо ідентифікатори датасету на ті, що згенерувало сховище (driver_id, constructor_id)

In [22]:
races_fp2_results_df['driver_id'] = races_fp2_results_df.merge(drivers_id_df, left_on='driverId', right_on='id', how='left')['dbId']
races_fp2_results_df['constructor_id'] = races_fp2_results_df.merge(constructors_id_df, left_on='constructorId', right_on='id', how='left')['dbId']

Приводимо дані у потрібний нам формат для сховища

In [23]:
races_fp2_results_df = races_fp2_results_df[['raceId', 'driver_id', 'constructor_id', 'positionDisplayOrder', 'positionNumber', 'time', 'timeMillis', 'gap', 'gapMillis', 'interval', 'intervalMillis', 'laps']]
races_fp2_results_df.head()

,raceId,driver_id,constructor_id,positionDisplayOrder,positionNumber,time,timeMillis,gap,gapMillis,interval,intervalMillis,laps
0,435,68,101,1,1,1:17.977,77977.0,NaN,NaN,NaN,NaN,NaN
1,435,316,22,2,2,1:18.088,78088.0,+0.111,111.0,+0.111,111.0,NaN
2,435,834,22,3,3,1:18.154,78154.0,+0.177,177.0,+0.066,66.0,NaN
3,435,644,180,4,4,1:18.353,78353.0,+0.376,376.0,+0.199,199.0,NaN
4,435,652,180,5,5,1:18.785,78785.0,+0.808,808.0,+0.432,432.0,NaN


Перейменуємо стовпці

In [24]:
races_fp2_results_df.columns = ['race_id', 'driver_id', 'constructor_id', 'position_display_order', 'position_number', 'time', 'time_millis', 'gap', 'gap_millis', 'interval', 'interval_millis', 'laps']

Вносимо дані з датафрейму у таблицю сховища

In [30]:
insert_data(races_fp2_results_df, 'races_fp2_results')

## Races FP3 Results Table

In [26]:
races_fp3_results_df = pd.read_csv('../data/f1db-races-free-practice-3-results.csv')
races_fp3_results_df.head()

,raceId,year,round,positionDisplayOrder,positionNumber,positionText,driverNumber,driverId,constructorId,engineManufacturerId,tyreManufacturerId,time,timeMillis,gap,gapMillis,interval,intervalMillis,laps
0,706,2003,9,1,1,1,4,ralf-schumacher,williams,bmw,michelin,1:31.305,91305.0,NaN,NaN,NaN,NaN,11
1,706,2003,9,2,2,2,3,juan-pablo-montoya,williams,bmw,michelin,1:31.366,91366.0,0.061,61.0,0.061,61.0,15
2,706,2003,9,3,3,3,20,olivier-panis,toyota,toyota,michelin,1:31.490,91490.0,0.185,185.0,0.124,124.0,18
3,706,2003,9,4,4,4,5,david-coulthard,mclaren,mercedes,michelin,1:31.608,91608.0,0.303,303.0,0.118,118.0,16
4,706,2003,9,5,5,5,6,kimi-raikkonen,mclaren,mercedes,michelin,1:32.021,92021.0,0.716,716.0,0.413,413.0,13


Замінимо ідентифікатори датасету на ті, що згенерувало сховище (driver_id, constructor_id)

In [27]:
races_fp3_results_df['driver_id'] = races_fp3_results_df.merge(drivers_id_df, left_on='driverId', right_on='id', how='left')['dbId']
races_fp3_results_df['constructor_id'] = races_fp3_results_df.merge(constructors_id_df, left_on='constructorId', right_on='id', how='left')['dbId']

Приводимо дані у потрібний нам формат для сховища

In [28]:
races_fp3_results_df = races_fp3_results_df[['raceId', 'driver_id', 'constructor_id', 'positionDisplayOrder', 'positionNumber', 'time', 'timeMillis', 'gap', 'gapMillis', 'interval', 'intervalMillis', 'laps']]
races_fp3_results_df.head()

,raceId,driver_id,constructor_id,positionDisplayOrder,positionNumber,time,timeMillis,gap,gapMillis,interval,intervalMillis,laps
0,706,736,180,1,1,1:31.305,91305.0,NaN,NaN,NaN,NaN,11
1,706,506,180,2,2,1:31.366,91366.0,0.061,61.0,0.061,61.0,15
2,706,664,169,3,3,1:31.490,91490.0,0.185,185.0,0.124,124.0,18
3,706,191,112,4,4,1:31.608,91608.0,0.303,303.0,0.118,118.0,16
4,706,530,112,5,5,1:32.021,92021.0,0.716,716.0,0.413,413.0,13


Перейменуємо стовпці

In [29]:
races_fp3_results_df.columns = ['race_id', 'driver_id', 'constructor_id', 'position_display_order', 'position_number', 'time', 'time_millis', 'gap', 'gap_millis', 'interval', 'interval_millis', 'laps']

Вносимо дані з датафрейму у таблицю сховища

In [32]:
insert_data(races_fp3_results_df, 'races_fp3_results')

## Races Qualifying Results Table

In [33]:
races_qualifying_results_df = pd.read_csv('../data/f1db-races-qualifying-results.csv')
races_qualifying_results_df.head()

,raceId,year,round,positionDisplayOrder,positionNumber,positionText,driverNumber,driverId,constructorId,engineManufacturerId,...,q1Millis,q2,q2Millis,q3,q3Millis,gap,gapMillis,interval,intervalMillis,laps
0,1,1950,1,1,1.0,1,2,nino-farina,alfa-romeo,alfa-romeo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1950,1,2,2.0,2,3,luigi-fagioli,alfa-romeo,alfa-romeo,...,NaN,NaN,NaN,NaN,NaN,+0.200,200.0,+0.200,200.0,NaN
2,1,1950,1,3,3.0,3,1,juan-manuel-fangio,alfa-romeo,alfa-romeo,...,NaN,NaN,NaN,NaN,NaN,+0.200,200.0,+0.000,0.0,NaN
3,1,1950,1,4,4.0,4,4,reg-parnell,alfa-romeo,alfa-romeo,...,NaN,NaN,NaN,NaN,NaN,+1.400,1400.0,+1.200,1200.0,NaN
4,1,1950,1,5,5.0,5,21,birabongse-bhanudej,maserati,maserati,...,NaN,NaN,NaN,NaN,NaN,+1.800,1800.0,+0.400,400.0,NaN


Замінимо ідентифікатори датасету на ті, що згенерувало сховище (driver_id, constructor_id)

In [34]:
races_qualifying_results_df['driver_id'] = races_qualifying_results_df.merge(drivers_id_df, left_on='driverId', right_on='id', how='left')['dbId']
races_qualifying_results_df['constructor_id'] = races_qualifying_results_df.merge(constructors_id_df, left_on='constructorId', right_on='id', how='left')['dbId']

Приводимо дані у потрібний нам формат для сховища

In [35]:
races_qualifying_results_df = races_qualifying_results_df[['raceId', 'driver_id', 'constructor_id', 'positionDisplayOrder', 'positionNumber', 'q1', 'q1Millis', 'q2', 'q2Millis', 'q3', 'q3Millis', 'gap', 'gapMillis', 'interval', 'intervalMillis', 'laps']]
races_qualifying_results_df.head()

,raceId,driver_id,constructor_id,positionDisplayOrder,positionNumber,q1,q1Millis,q2,q2Millis,q3,q3Millis,gap,gapMillis,interval,intervalMillis,laps
0,1,655,4,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,565,4,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,+0.200,200.0,+0.200,200.0,NaN
2,1,505,4,3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,+0.200,200.0,+0.000,0.0,NaN
3,1,742,4,4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,+1.400,1400.0,+1.200,1200.0,NaN
4,1,93,109,5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,+1.800,1800.0,+0.400,400.0,NaN


Перейменуємо стовпці

In [36]:
races_qualifying_results_df.columns = ['race_id', 'driver_id', 'constructor_id', 'position_display_order', 'position_number', 'q1', 'q1_millis', 'q2', 'q2_millis', 'q3', 'q3_millis', 'gap', 'gap_millis', 'interval', 'interval_millis', 'laps']

Вносимо дані з датафрейму у таблицю сховища

In [37]:
insert_data(races_qualifying_results_df, 'races_qualifying_results')